## Predict the next vehicle state given a sequence of vehicle controls and the current vehicle state.

    f(control, current_state) -> next_state

In [13]:
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


In [14]:
AHRS = pd.read_csv('ahrs.csv')
CONTROL = pd.read_csv('vehicle_control.csv')

AHRS['ts'] = pd.to_datetime(AHRS['ts'])
CONTROL['ts'] = pd.to_datetime(CONTROL['ts'])
merged_df = pd.merge_asof(AHRS,CONTROL, on='ts',direction='backward').dropna()
# control_cols = list(control_df.drop('ts', axis=1).columns)
# AHRS_COLS = list(state_df.drop('ts', axis=1).columns)
AHRS_COLS = list(AHRS.columns.drop('ts'))
NEXT_AHRS_COLS = [col+"_next" for col in AHRS_COLS]
CONTROL_COLS = list(CONTROL.columns.drop('ts'))
print(CONTROL_COLS)
print(AHRS_COLS)

['gear', 'throttle', 'trim', 'turn']
['roll_deg', 'pitch_deg', 'yaw_deg', 'ax_mps2', 'ay_mps2', 'az_mps2', 'omega_x_dps', 'omega_y_dps', 'omega_z_dps', 've_mps', 'vn_mps', 'vu_mps']


In [15]:
display(merged_df.head())

,ts,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps,gear,throttle,trim,turn
1,2024-07-31 14:05:00.092972588+00:00,0.071586,-1.813645,340.940125,-9.969663,0.094937,-0.338713,-0.11750,-0.10375,-0.44750,-0.000357,0.000355,0.002445,0.0,0.008547,2.0,0.003663
2,2024-07-31 14:05:00.113763246+00:00,0.070876,-1.813810,340.938721,-9.963537,0.140262,-0.315438,-0.05125,-0.22125,-0.41750,-0.000010,0.000226,0.000702,0.0,0.008547,2.0,0.003663
3,2024-07-31 14:05:00.133055196+00:00,0.070295,-1.813962,340.939148,-9.992325,0.100450,-0.323400,-0.04250,-0.17250,-0.39125,-0.000253,0.000390,0.000410,0.0,0.008547,2.0,0.003663
4,2024-07-31 14:05:00.153150190+00:00,0.070045,-1.814164,340.939636,-10.025400,0.109637,-0.335650,-0.14375,-0.17375,-0.36750,-0.000029,0.000404,0.000288,0.0,0.008547,2.0,0.003663
5,2024-07-31 14:05:00.172988544+00:00,0.070483,-1.814484,340.939484,-10.099512,0.073500,-0.355863,-0.13750,-0.19625,-0.42375,0.000196,0.000485,0.000386,0.0,0.008547,2.0,0.003663


So by inspection from 'explorer', we see a lot of the controls are duplicate commands. Meaning, that between state n and state n+1, the controls to get to state n+1 are largely homogenous. This will guide my model selection 

In [16]:
for col in AHRS_COLS:
    merged_df[col + "_next"] = merged_df[col].shift(-1)
merged_df = merged_df.dropna()
merged_df.head()

,ts,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps,gear,throttle,trim,turn,roll_deg_next,pitch_deg_next,yaw_deg_next,ax_mps2_next,ay_mps2_next,az_mps2_next,omega_x_dps_next,omega_y_dps_next,omega_z_dps_next,ve_mps_next,vn_mps_next,vu_mps_next
1,2024-07-31 14:05:00.092972588+00:00,0.071586,-1.813645,340.940125,-9.969663,0.094937,-0.338713,-0.11750,-0.10375,-0.44750,-0.000357,0.000355,0.002445,0.0,0.008547,2.0,0.003663,0.070876,-1.813810,340.938721,-9.963537,0.140262,-0.315438,-0.05125,-0.22125,-0.41750,-0.000010,0.000226,0.000702
2,2024-07-31 14:05:00.113763246+00:00,0.070876,-1.813810,340.938721,-9.963537,0.140262,-0.315438,-0.05125,-0.22125,-0.41750,-0.000010,0.000226,0.000702,0.0,0.008547,2.0,0.003663,0.070295,-1.813962,340.939148,-9.992325,0.100450,-0.323400,-0.04250,-0.17250,-0.39125,-0.000253,0.000390,0.000410
3,2024-07-31 14:05:00.133055196+00:00,0.070295,-1.813962,340.939148,-9.992325,0.100450,-0.323400,-0.04250,-0.17250,-0.39125,-0.000253,0.000390,0.000410,0.0,0.008547,2.0,0.003663,0.070045,-1.814164,340.939636,-10.025400,0.109637,-0.335650,-0.14375,-0.17375,-0.36750,-0.000029,0.000404,0.000288
4,2024-07-31 14:05:00.153150190+00:00,0.070045,-1.814164,340.939636,-10.025400,0.109637,-0.335650,-0.14375,-0.17375,-0.36750,-0.000029,0.000404,0.000288,0.0,0.008547,2.0,0.003663,0.070483,-1.814484,340.939484,-10.099512,0.073500,-0.355863,-0.13750,-0.19625,-0.42375,0.000196,0.000485,0.000386
5,2024-07-31 14:05:00.172988544+00:00,0.070483,-1.814484,340.939484,-10.099512,0.073500,-0.355863,-0.13750,-0.19625,-0.42375,0.000196,0.000485,0.000386,0.0,0.008547,2.0,0.003663,0.070457,-1.814585,340.939148,-10.113600,0.088812,-0.332588,-0.11000,-0.19125,-0.46625,0.000439,0.000571,0.000511


In [17]:
def extractRaw(df):
    X_state = df[AHRS_COLS].to_numpy()
    X_control = df[CONTROL_COLS].to_numpy()
    y_next_state = df[NEXT_AHRS_COLS].to_numpy()
    return X_state, X_control, y_next_state

X_state, X_control, y_next_state = extractRaw(merged_df)
X_full = np.hstack([X_state, X_control])
print(X_full.shape)
testcols = AHRS_COLS + CONTROL_COLS 
print(testcols)
GEAR_IDX = testcols.index('gear')
TRIM_IDX = testcols.index('trim')
print(GEAR_IDX, TRIM_IDX)
print(X_full[0,:])


(344999, 16)
['roll_deg', 'pitch_deg', 'yaw_deg', 'ax_mps2', 'ay_mps2', 'az_mps2', 'omega_x_dps', 'omega_y_dps', 'omega_z_dps', 've_mps', 'vn_mps', 'vu_mps', 'gear', 'throttle', 'trim', 'turn']
12 14
[ 7.15858936e-02 -1.81364465e+00  3.40940125e+02 -9.96966250e+00
  9.49375000e-02 -3.38712500e-01 -1.17500000e-01 -1.03750000e-01
 -4.47500000e-01 -3.56881181e-04  3.54545307e-04  2.44454597e-03
  0.00000000e+00  8.54700900e-03  2.00000000e+00  3.66300370e-03]


before scaling:

az_mps2 might range from -30 to +30
ve_mps might range from 0 to 5

after scaling:
Both features will be centered around 0 and have similar ranges

In [18]:
class MultiHeadModel(nn.Module):
    def __init__(self, input_dim=20):
        super().__init__()
        self.combo = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
        )

        self.head_vel = nn.Linear(128, 3)
        self.head_acc = nn.Sequential(nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 3))
        self.head_ang = nn.Linear(128, 3)
        self.head_omega = nn.Linear(128, 3)

        self.path = "AHRS Models/state_predictor.pt"
        self.X_scaler_path = "AHRS Models/X_scaler.pkl"
        self.y_scaler_path = "AHRS Models/y_scaler.pkl"
        self.X_scaler = None
        self.y_scaler = None  
        self.gear_idx = 12
        self.trim_idx = 14

    def forward(self, x):
        combo = self.combo(x)
        return torch.cat([
            self.head_vel(combo),
            self.head_acc(combo),
            self.head_ang(combo),
            self.head_omega(combo),
        ], dim=1)

    def one_hot_encode_np(self, X):
        assert X.shape[1] == 16
        gear = X[:, self.gear_idx].astype(int)
        trim = X[:, self.trim_idx].astype(int)

        gear_encoded = np.eye(3)[gear]  # shape (N, 3)
        trim_encoded = np.eye(3)[trim]  # shape (N, 3)

        # Remove gear and trim columns
        X_base = np.delete(X, [self.gear_idx, self.trim_idx], axis=1)

        # Concatenate all together
        X_encoded = np.hstack([X_base, gear_encoded, trim_encoded])
        return X_encoded

    def fit(self, X_train, y_train, epochs=20, batch_size=64, lr=1e-3):
        os.makedirs(os.path.dirname(self.path), exist_ok=True)

        # One-hot encode gear and trim
        X_train = self.one_hot_encode_np(X_train)
        if os.path.exists(self.X_scaler_path) and os.path.exists(self.y_scaler_path): 
            print("Loading previously saved scalers...")
            self.X_scaler = joblib.load(self.X_scaler_path)
            self.y_scaler = joblib.load(self.y_scaler_path)
        else:
            print("Fitting and saving new scalers...")
            self.X_scaler = StandardScaler().fit(X_train)
            self.y_scaler = StandardScaler().fit(y_train)
            joblib.dump(self.X_scaler, self.X_scaler_path)
            joblib.dump(self.y_scaler, self.y_scaler_path)

        # Scale input and output
        X_scaled = self.X_scaler.transform(X_train)
        y_scaled = self.y_scaler.transform(y_train)

        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Train or load model
        if not os.path.exists(self.path):
            self.train()
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
            loss_fn = nn.MSELoss()

            self.history = []

            for epoch in tqdm(range(epochs), "Training..."):
                total_loss = 0
                for batch_inputs, batch_targets in loader:
                    optimizer.zero_grad()  # Reset gradients
                    predictions = self(batch_inputs)  
                    loss = loss_fn(predictions, batch_targets) 
                    loss.backward()  
                    optimizer.step()  
                    total_loss += loss.item()  

                avg_loss = total_loss / len(loader)
                self.history.append(avg_loss)
                print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

            torch.save(self.state_dict(), self.path)
        else:
            print("Loading previously saved model. Delete AHRS Models to start fresh.")
            self.load_state_dict(torch.load(self.path))

    def predict(self, X):
        self.load_state_dict(torch.load(self.path))
        self.eval()

        if self.X_scaler is None or self.y_scaler is None:
            self.X_scaler = joblib.load(self.X_scaler_path)
            self.y_scaler = joblib.load(self.y_scaler_path)

        # One-hot encode gear and trim
        X = self.one_hot_encode_np(X)
        # Scale
        X_scaled = self.X_scaler.transform(X)
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

        with torch.no_grad():
            preds_scaled = self(X_tensor).numpy()

        preds = self.y_scaler.inverse_transform(preds_scaled)
        return preds

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_next_state, test_size=0.3)
model = MultiHeadModel()
print(X_train.shape[1])
model.fit(X_train, y_train)

16
Loading previously saved scalers...
Loading previously saved model. Delete AHRS Models to start fresh.


Below is to verify by inspection the one hot looks as expected

In [20]:
sample = X_train[0:5, :]  # shape: (5, 16)
print("Gear and Trim values (raw):")
for i in range(sample.shape[0]):
    print(f"Row {i}: gear = {sample[i][12]}, trim = {sample[i][14]}")

print("Raw shape:", sample.shape)

# One-hot encode
sample_encoded = model.one_hot_encode_np(sample)

# Print the one-hot parts (last 6 columns) for each row
print("\nOne-hot encoded gear + trim:")
for i in range(sample_encoded.shape[0]):
    print(f"Row {i}: {sample_encoded[i][-6:]}")

print("Encoded shape:", sample_encoded.shape)


Gear and Trim values (raw):
Row 0: gear = 0.0, trim = 0.0
Row 1: gear = 1.0, trim = 0.0
Row 2: gear = 1.0, trim = 0.0
Row 3: gear = 1.0, trim = 0.0
Row 4: gear = 1.0, trim = 0.0
Raw shape: (5, 16)

One-hot encoded gear + trim:
Row 0: [1. 0. 0. 1. 0. 0.]
Row 1: [0. 1. 0. 1. 0. 0.]
Row 2: [0. 1. 0. 1. 0. 0.]
Row 3: [0. 1. 0. 1. 0. 0.]
Row 4: [0. 1. 0. 1. 0. 0.]
Encoded shape: (5, 20)


In [21]:
def predict_next_state(state_raw, control_raw):
    X = np.hstack([state_raw, control_raw]).reshape(1, -1)
    #print(X.shape)
    val = np.array(model.predict(X))
    return val


In [22]:
def compare_prediction_error_np(actual_array, predicted_array, feature_names=None, sig_figs=None):
    # print(actual_array.shape)
    # print(predicted_array.shape)
    predicted_array = predicted_array.flatten()
    assert actual_array.shape == predicted_array.shape, "Shapes must match"
    assert actual_array.ndim == 1, "Inputs must be 1D arrays (single sample)"

    # Compute errors
    abs_error = np.abs(actual_array - predicted_array)
    percent_error = (abs_error / (np.abs(actual_array) + 1e-8)) * 100

    # Default column names
    if feature_names is None:
        feature_names = [l for l in AHRS_COLS]

    # Create DataFrame
    comparison_df = pd.DataFrame({
        'Actual': actual_array,
        'Predicted': predicted_array,
        'Abs Error': abs_error,
        '% Error': percent_error
    }, index=feature_names)

    if sig_figs is not None:
        comparison_df = comparison_df.round(sig_figs)

    return comparison_df


Run the cell below to compare a sample comparsion side-by-side

In [23]:
sample = merged_df.sample(n=1)
current_state = sample[AHRS_COLS].values[0]
next_state = sample[NEXT_AHRS_COLS].values[0]
control = sample[CONTROL_COLS].values[0]
prediction = predict_next_state(current_state,control)
comparison = compare_prediction_error_np(next_state, prediction)
display(comparison)

,Actual,Predicted,Abs Error,% Error
roll_deg,-0.693614,-0.646085,0.047529,6.852393
pitch_deg,4.986008,5.048151,0.062142,1.246335
yaw_deg,227.798477,217.988632,9.809845,4.306370
ax_mps2,-11.457425,-11.302036,0.155389,1.356227
ay_mps2,-1.218263,-0.591228,0.627035,51.469576
az_mps2,1.169262,0.666445,0.502817,43.002949
omega_x_dps,2.251250,1.921228,0.330022,14.659519
omega_y_dps,-3.305000,-3.052484,0.252516,7.640423
omega_z_dps,0.316250,1.501807,1.185557,374.879702
ve_mps,-5.630569,-5.280752,0.349817,6.212815


# Performance Plots for the Test Set

In [24]:
os.makedirs("plots",exist_ok=True)
os.makedirs('plots/AHRS Plots',exist_ok=True)
preds_all = model.predict(X_test)  
for i, col in enumerate(AHRS_COLS):
    actual = y_test[:,i]
    preds = preds_all[:,i]
    assert actual.shape == preds.shape
    plt.figure(figsize=(6, 6))
    plt.scatter(actual, preds, alpha=0.6, color='navy')
    plt.plot([actual.min(), actual.max()],[actual.min(), actual.max()],linestyle='--',label='Correct',color='gold')
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.title(f"{col} - Actual vs Predicted Scatter (Test Set)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"plots/AHRS Plots/{col}_scatter.png")
    plt.close()